In [40]:
import numpy as np

import torch
from torchvision import models, transforms, datasets
from torch.utils.data import DataLoader
import torch.nn.functional as F
import math

from torch import nn

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('Current device is', device)

Current device is cpu


In [2]:
train_set = datasets.FashionMNIST('data/homework1/', 
                           download=True, 
                           train=True, 
                           transform=transforms.Compose([transforms.ToTensor()]))
test_set = datasets.FashionMNIST('data/homework1/', 
                          download=True, 
                          train=False, 
                          transform=transforms.Compose([transforms.ToTensor()]))

In [3]:
print('Train data size: ', train_set.train_data.size())
print('Train data labels: ', train_set.train_labels.size())

print('Test data size: ', test_set.test_data.size())
print('Test data labels: ', test_set.test_labels.size())

Train data size:  torch.Size([60000, 28, 28])
Train data labels:  torch.Size([60000])
Test data size:  torch.Size([10000, 28, 28])
Test data labels:  torch.Size([10000])


/Users/zhuvikin/anaconda3/envs/AIU_ML/lib/python3.7/site-packages/torchvision/datasets/mnist.py:53: UserWarning: train_data has been renamed data
  warnings.warn("train_data has been renamed data")
/Users/zhuvikin/anaconda3/envs/AIU_ML/lib/python3.7/site-packages/torchvision/datasets/mnist.py:43: UserWarning: train_labels has been renamed targets
  warnings.warn("train_labels has been renamed targets")
/Users/zhuvikin/anaconda3/envs/AIU_ML/lib/python3.7/site-packages/torchvision/datasets/mnist.py:58: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")
/Users/zhuvikin/anaconda3/envs/AIU_ML/lib/python3.7/site-packages/torchvision/datasets/mnist.py:48: UserWarning: test_labels has been renamed targets
  warnings.warn("test_labels has been renamed targets")


In [41]:
lr = 0.001
epochs = 5
batch_size = 100

In [42]:
train_loader = torch.utils.data.DataLoader(train_set, shuffle=True, batch_size=batch_size)
test_loader = torch.utils.data.DataLoader(test_set, shuffle=False, batch_size=batch_size)

In [43]:
def train_model(model, data_loader, device, epochs = epochs):
    optimizer = torch.optim.Adam(model.parameters(), lr = lr)
    criterion = nn.CrossEntropyLoss()

    model.train()

    for e in range(epochs):
        total_loss = 0
        for batch, (target, labels) in enumerate(train_loader):
            target = target.to(device)
            labels = labels.to(device)

            # Zero the gradients before running the backwprop.
            optimizer.zero_grad()

            yhat = model(target)

            loss = criterion(yhat, labels)

            # Computer all the gradients of the loss in the respect to all learnable params
            loss.backward()

            # Update the weight
            
            optimizer.step()
            total_loss += loss.item()

            if batch % math.ceil(8000 / batch_size) == math.ceil(8000 / batch_size) - 1:
                print('Epoch {}, Train loss {}'.format(e, total_loss / 2000))
                total_loss = 0
    return model

In [44]:
class NeuralNetwork(nn.Module):
    
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        
        self.fc1 = nn.Linear(28*28, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 10)
        
    def forward(self, x):
        x = x.view(-1, 28*28)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [45]:
model = train_model(NeuralNetwork().to(device), train_loader, device)

Epoch 0, Train loss 0.045375131905078886
Epoch 0, Train loss 0.025201572000980377
Epoch 0, Train loss 0.02250900074839592
Epoch 0, Train loss 0.02062866860628128
Epoch 0, Train loss 0.019679408699274065
Epoch 0, Train loss 0.018786437667906285
Epoch 0, Train loss 0.01779517535865307
Epoch 1, Train loss 0.01732957324385643
Epoch 1, Train loss 0.01691229233145714
Epoch 1, Train loss 0.016797538504004478
Epoch 1, Train loss 0.016515687480568884
Epoch 1, Train loss 0.015874560214579104
Epoch 1, Train loss 0.015959055319428445
Epoch 1, Train loss 0.01600893058627844
Epoch 2, Train loss 0.015009997375309468
Epoch 2, Train loss 0.01591036770492792
Epoch 2, Train loss 0.015225421376526356
Epoch 2, Train loss 0.013897190295159817
Epoch 2, Train loss 0.015044055074453354
Epoch 2, Train loss 0.014503111205995083
Epoch 2, Train loss 0.014564092881977558
Epoch 3, Train loss 0.013944836474955083
Epoch 3, Train loss 0.013663594134151935
Epoch 3, Train loss 0.013572278261184692
Epoch 3, Train loss 0.0

In [46]:
def test_model(model, dataLoader):
    model.eval()
    test_loss = 0
    correct = 0

    with torch.no_grad():
        for target, labels in dataLoader:
            target, labels = target.to(device), labels.to(device)
            yhat = model(target)
            test_loss += F.nll_loss(yhat, labels, reduction='sum').item()
            prediction = yhat.argmax(dim=1, keepdim=True)
            correct += prediction.eq(labels.view_as(prediction)).sum().item()

    print('Test set: Accuracy: {}/{} ({:.0f}%)\n'.format(
        correct, len(dataLoader.dataset), 100. * correct / len(dataLoader.dataset)))

In [47]:
test_model(model, test_loader)

Test set: Accuracy: 8686/10000 (87%)

